In [1]:
import sys
sys.path.append('..')

In [2]:
from aimnet import load_AIMNetMT_ens, load_AIMNetSMD_ens, AIMNetCalculator
import pybel
import nglview
import ase
import ase.optimize
import ase.md
import numpy as np

_ColormakerRegistry()

In [3]:
def pybel2ase(mol):
    coord = np.asarray([a.coords for a in mol.atoms])
    numbers = np.asarray([a.atomicnum for a in mol.atoms])
    return ase.Atoms(positions=coord, numbers=numbers)

### Construct molecule from SMILES

In [4]:
smiles = 'O=C([O-])[C@@H]([NH3+])Cc1c[nH]cn1'  # Histidine
mol = pybel.readstring('smi', smiles)
mol.make3D()
nglview.show_openbabel(mol.OBMol)

NGLWidget()

### Load pre-trained esnsembles of AIMNet models

load_AIMNetMT_ens - loads a model trained to wB97x/def2-TZVPP energies, atomic electric moments (charges, dipoles, etc) and volumes

load_AIMNetSMD_ens - loads a model obtained by transfer learning towards SMD-wB97x/def2-TZVPP energies

In [5]:
model_gas = load_AIMNetMT_ens().cuda()
model_smd = load_AIMNetSMD_ens().cuda()

calc_gas = AIMNetCalculator(model_gas)
calc_smd = AIMNetCalculator(model_smd)

### Run geometry optimization in gas phase

Use slider to look through optimization trajectory. Notice proton transfer from N to O. 

In [6]:
atoms = pybel2ase(mol)

atoms.set_calculator(calc_gas)
opt = ase.optimize.BFGS(atoms, trajectory='gas_opt.traj')
opt.run(0.02)

traj = ase.io.Trajectory('gas_opt.traj', 'r')
nglview.show_asetraj(traj)

      Step     Time          Energy         fmax
BFGS:    0 12:33:33   -14933.829426        2.0158
BFGS:    1 12:33:33   -14933.950250        1.2122
BFGS:    2 12:33:33   -14933.986133        0.4958
BFGS:    3 12:33:33   -14934.004384        0.2661
BFGS:    4 12:33:33   -14934.011528        0.2922
BFGS:    5 12:33:33   -14934.019828        0.3345
BFGS:    6 12:33:33   -14934.029376        0.3836
BFGS:    7 12:33:33   -14934.040973        0.3911
BFGS:    8 12:33:33   -14934.052617        0.2792
BFGS:    9 12:33:34   -14934.063498        0.2781
BFGS:   10 12:33:34   -14934.074082        0.3338
BFGS:   11 12:33:34   -14934.085429        0.2928
BFGS:   12 12:33:34   -14934.095969        0.2852
BFGS:   13 12:33:34   -14934.107179        0.2425
BFGS:   14 12:33:34   -14934.116733        0.2780
BFGS:   15 12:33:34   -14934.124243        0.2797
BFGS:   16 12:33:34   -14934.131303        0.2820
BFGS:   17 12:33:34   -14934.139999        0.3081
BFGS:   18 12:33:34   -14934.149496        0.2997
B

NGLWidget(max_frame=160)

#### Get atomic charges and volumes

In [7]:
charges = calc_gas.results['elmoments'][0, :, 0]
print('Charges: ', charges)

volumes = calc_gas.results['volume'][0]
print('Volumes: ', volumes)

Charges:  [-0.62805146  0.76900166 -0.58449894 -0.01507504  0.12756534 -0.92480814
  0.4135644   0.3693852   0.47252727 -0.3434528   0.27139837 -0.2139298
 -0.38574287  0.36914736  0.25277984 -0.5525247   0.16195168  0.15864624
  0.17289384  0.11310641]
Volumes:  [25.58634   21.452873  25.87126   32.231163   2.7129638 37.92253
  1.324596   1.5256053  1.3165376 36.787827  27.13393   35.796875
 28.835033   1.4570949 28.413046  33.059605   2.5888383  2.637519
  2.5069685  2.8412147]


### Run geometry optimization with SMD explicit solvation correction

Amino acid stays in zwitter-ionic form.

In [8]:
atoms = pybel2ase(mol)

atoms.set_calculator(calc_smd)
opt = ase.optimize.BFGS(atoms, trajectory='smd_opt.traj')
opt.run(0.02)

traj = ase.io.Trajectory('smd_opt.traj', 'r')
nglview.show_asetraj(traj)

      Step     Time          Energy         fmax
BFGS:    0 12:33:45   -14935.061359        1.7385
BFGS:    1 12:33:45   -14935.191080        1.1922
BFGS:    2 12:33:45   -14935.246467        0.5179
BFGS:    3 12:33:45   -14935.261545        0.2743
BFGS:    4 12:33:45   -14935.268252        0.2709
BFGS:    5 12:33:45   -14935.275128        0.2208
BFGS:    6 12:33:45   -14935.282826        0.3342
BFGS:    7 12:33:45   -14935.288899        0.2419
BFGS:    8 12:33:45   -14935.293318        0.1632
BFGS:    9 12:33:45   -14935.297417        0.1525
BFGS:   10 12:33:45   -14935.301456        0.1912
BFGS:   11 12:33:46   -14935.305073        0.1498
BFGS:   12 12:33:46   -14935.308269        0.1281
BFGS:   13 12:33:46   -14935.311389        0.1297
BFGS:   14 12:33:46   -14935.314545        0.1613
BFGS:   15 12:33:46   -14935.317840        0.1404
BFGS:   16 12:33:46   -14935.321305        0.1489
BFGS:   17 12:33:46   -14935.324616        0.1403
BFGS:   18 12:33:46   -14935.327267        0.1153
B

NGLWidget(max_frame=132)